In [4]:
import hashlib
import json
import time

### Block

The Block class represents a single block in the blockchain. It contains information about its index, transactions, timestamp, previous hash, nonce, and hash. It has a method calculate_hash() to compute the SHA-256 hash of the block, and a method mine_block() to mine the block by finding a nonce that results in a hash with a certain number of leading zeros. The number of leading zeros is determined by the difficulty of the blockchain. The block is mined by repeatedly calling calculate_hash() and incrementing the nonce until the hash has the required number of leading zeros. The block is then added to the blockchain.

In [5]:
class Block:
    def __init__(self, index, transactions, timestamp, previous_hash):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = json.dumps({
            "index": self.index,
            "transactions": self.transactions,
            "timestamp": self.timestamp,
            "previous_hash": self.previous_hash,
            "nonce": self.nonce,
        }, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def mine_block(self, difficulty):
        while self.hash[:difficulty] != "0" * difficulty:
            self.nonce += 1
            self.hash = self.calculate_hash()
        print("Block mined:", self.hash)

### Blockchain

The Blockchain class represents the entire blockchain network. It contains a list of blocks, a difficulty level for mining, a list of pending transactions, and a mining reward. It has methods to create the genesis block, get the latest block, add a transaction to the pending transactions list, mine the pending transactions into a new block, and get the balance of a given address.

In [6]:
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.difficulty = 2
        self.pending_transactions = []
        self.mining_reward = 100

    def create_genesis_block(self):
        return Block(0, [], time.time(), "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_transaction(self, sender, recipient, amount):
        self.pending_transactions.append({
            "sender": sender,
            "recipient": recipient,
            "amount": amount,
        })

    def mine_pending_transactions(self, miner_address):
        block = Block(len(self.chain), self.pending_transactions, time.time(), self.get_latest_block().hash)
        block.mine_block(self.difficulty)
        self.chain.append(block)
        self.pending_transactions = []
        self.add_transaction(None, miner_address, self.mining_reward)

    def get_balance(self, address):
        balance = 0
        for block in self.chain:
            for transaction in block.transactions:
                if transaction["sender"] == address:
                    balance -= transaction["amount"]
                elif transaction["recipient"] == address:
                    balance += transaction["amount"]
        return balance

### Test

In [7]:
blockchain = Blockchain()

blockchain.add_transaction("Alice", "Bob", 10)
blockchain.add_transaction("Bob", "Charlie", 5)

blockchain.mine_pending_transactions("Miner")

print("Alice's balance:", blockchain.get_balance("Alice"))
print("Bob's balance:", blockchain.get_balance("Bob"))
print("Charlie's balance:", blockchain.get_balance("Charlie"))
print("Miner's balance:", blockchain.get_balance("Miner"))

Block mined: 0048bc0dc5a922bc58e2c0dbebf71b47ea25dbf0def202955ef54bf6fc2a6be2
Alice's balance: -10
Bob's balance: 5
Charlie's balance: 5
Miner's balance: 0
